<a href="https://colab.research.google.com/github/antussa2016/thesis_work/blob/master/model/thesis_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! git clone https://github.com/antussa2016/thesis_work

fatal: destination path 'thesis_work' already exists and is not an empty directory.


In [3]:
import matplotlib.pyplot as plt
import pandas as pd

In [16]:
df = pd.read_csv('/content/thesis_work/dataset/final_data_for_modeling .csv')

In [17]:
df.head(1)

,twitter_id,stackoverflow_id,question_count,answer_count,reputation,answer_tag,question_tag,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,preferred_tags1,preferred_tags2,preferred_tags3,oop,web-pro,database,mobile
0,10F1,145587,13,500,74297,"['go', 'json', 'http', 'goroutine', 'struct', ...","['go', 'javascript', 'python', 'c++', 'linux',...",0.839299,0.170988,0.083984,0.047367,0.612101,0,json,0,0,1,0,0


In [18]:
df.drop(columns=['twitter_id','stackoverflow_id','question_count','reputation','answer_tag','question_tag','Extraversion'], inplace=True)

In [19]:
df.head(1)

,answer_count,Openness,Conscientiousness,Agreeableness,Neuroticism,preferred_tags1,preferred_tags2,preferred_tags3,oop,web-pro,database,mobile
0,500,0.839299,0.170988,0.047367,0.612101,0,json,0,0,1,0,0


In [20]:
# Calculate min and max values for the column
min_value = df['answer_count'].min()
max_value = df['answer_count'].max()

# Apply min-max scaling to the column
df['answer_count'] = (df[['answer_count']] - min_value) / (max_value - min_value)

df.head(1)

,answer_count,Openness,Conscientiousness,Agreeableness,Neuroticism,preferred_tags1,preferred_tags2,preferred_tags3,oop,web-pro,database,mobile
0,1.0,0.839299,0.170988,0.047367,0.612101,0,json,0,0,1,0,0


In [ ]:
# # Drop rows where any row contains 0 in 'Column1'
# df = df[df['preferred_tags1'] != '0']


In [ ]:
# df = df[df['preferred_tags2'] != '0']

In [ ]:
# df = df[df['preferred_tags3'] != '0']

In [ ]:
# tag_to_code = {
#     '.net': 1,
#     'c#': 2,
#     'f#': 3,
#     'asp.net': 4,
#     'vb.net': 5,
#     'xml': 6,
#     'java': 7,
#     'python': 8,
#     'swing': 9,
#     'c++': 10,
#     'c': 11,
#     'php': 12,
#     'ruby': 13,
#     'perl': 14,
#     'jquery': 15,
#     'javascript': 16,
#     'node.js': 17,
#     'angularjs': 18,
#     'reactjs': 19,
#     'typescript': 20,
#     'html': 21,
#     'css': 22,
#     'json': 23,
#     'django': 24,
#     'sql': 25,
#     'sql-server': 26,
#     'mysql': 27,
#     'kotlin': 28,
#     'objective-c': 29,
#     'swift': 30,
#     'android': 31,
#     'ios': 32,
#     'go': 33,
#     'scala': 34,
#     'r': 35,
#     'git': 36,
#     'github': 38,
#     'os': 39,
#     'host': 40,
#     '0': 0


# }

In [ ]:
# df['preferred_tags1'] = df['preferred_tags1'].map(tag_to_code)
# df['preferred_tags2'] = df['preferred_tags2'].map(tag_to_code)
# df['preferred_tags3'] = df['preferred_tags3'].map(tag_to_code)

In [ ]:
# # Calculate min and max values for the column
# min_value = df['preferred_tags1'].min()
# max_value = df['preferred_tags1'].max()

# # Apply min-max scaling to the column
# df['preferred_tags1'] = (df[['preferred_tags1']] - min_value) / (max_value - min_value)


In [ ]:
# min_value = df['preferred_tags2'].min()
# max_value = df['preferred_tags2'].max()

# # Apply min-max scaling to the column
# df['preferred_tags2'] = (df[['preferred_tags2']] - min_value) / (max_value - min_value)

In [11]:
# min_value = df['preferred_tags3'].min()
# max_value = df['preferred_tags3'].max()

# # Apply min-max scaling to the column
# df['preferred_tags3'] = (df[['preferred_tags3']] - min_value) / (max_value - min_value)

In [43]:
import numpy as np
x= np.array(df.iloc[:,1:5])
y = np.array(df.iloc[:,8:12])


In [39]:
df.iloc[:,1:5]

,Openness,Conscientiousness,Agreeableness,Neuroticism
0,0.839299,0.170988,0.047367,0.612101
1,0.849555,0.100009,0.001654,0.392036
2,0.894954,0.124989,0.009011,0.288220
3,0.689216,0.625448,0.037679,0.223963
4,0.881779,0.042616,0.003885,0.432516
...,...,...,...,...
814,0.888007,0.739341,0.037764,0.342692
815,0.596079,0.105716,0.245279,1.000000
816,0.969066,0.055010,0.028997,0.295512
817,0.917028,0.062395,0.000687,0.461590


In [42]:
df.iloc[:,8:12]

,oop,web-pro,database,mobile
0,0,1,0,0
1,1,0,0,0
2,1,1,0,0
3,1,0,0,0
4,1,1,0,0
...,...,...,...,...
814,0,1,0,0
815,1,1,0,0
816,1,1,0,0
817,1,1,0,0


In [44]:
y = y.argmax(axis=1)

In [45]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [46]:
y_train.shape


(655,)

In [47]:
# Initialize and train an SVM classifier with hyperparameter tuning
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear']}
grid_search = GridSearchCV(SVC(), param_grid, cv=3)
grid_search.fit(X_train, y_train)


best_model = grid_search.best_estimator_


predictions = best_model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.6280487804878049


In [48]:
for i, (pred_label, true_label) in enumerate(zip(predictions, y_test)):
    print(f"Instance {i + 1}:")
    print("Predicted label:", pred_label)
    print("True label:", true_label)
    print("=" * 40)

Instance 1:
Predicted label: 0
True label: 1
Instance 2:
Predicted label: 0
True label: 0
Instance 3:
Predicted label: 0
True label: 0
Instance 4:
Predicted label: 0
True label: 1
Instance 5:
Predicted label: 0
True label: 0
Instance 6:
Predicted label: 0
True label: 0
Instance 7:
Predicted label: 0
True label: 1
Instance 8:
Predicted label: 0
True label: 2
Instance 9:
Predicted label: 0
True label: 0
Instance 10:
Predicted label: 0
True label: 1
Instance 11:
Predicted label: 0
True label: 0
Instance 12:
Predicted label: 0
True label: 1
Instance 13:
Predicted label: 0
True label: 1
Instance 14:
Predicted label: 0
True label: 0
Instance 15:
Predicted label: 0
True label: 0
Instance 16:
Predicted label: 0
True label: 0
Instance 17:
Predicted label: 0
True label: 0
Instance 18:
Predicted label: 0
True label: 0
Instance 19:
Predicted label: 0
True label: 0
Instance 20:
Predicted label: 0
True label: 1
Instance 21:
Predicted label: 0
True label: 0
Instance 22:
Predicted label: 0
True label:

# **Random forest**

In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
rf_classifier = RandomForestClassifier()
param_grid = {
   'n_estimators': [100, 200, 300],
   'max_depth': [None, 10, 20, 30],
   'min_samples_split': [2, 5, 10],
   'min_samples_leaf': [1, 2, 4],
   'max_features': ['auto', 'sqrt', 'log2'],
   'bootstrap': [True, False]
}
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=100, cv=5, n_jobs=-1, verbose=2, random_state=42)
random_search.fit(X_train, y_train)
best_rf_classifier = random_search.best_estimator_

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [50]:
from sklearn.metrics import classification_report
best_rf_classifier.fit(X_train, y_train)
y_pred = best_rf_classifier.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
report = classification_report(y_test, y_pred)
# print("report:", report)

Accuracy: 0.6402439024390244


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [51]:
for i, (pred_label, true_label) in enumerate(zip(predictions, y_test)):
    print(f"Instance {i + 1}:")
    print("Predicted label:", pred_label)
    print("True label:", true_label)
    print("=" * 40)

Instance 1:
Predicted label: 0
True label: 1
Instance 2:
Predicted label: 0
True label: 0
Instance 3:
Predicted label: 0
True label: 0
Instance 4:
Predicted label: 0
True label: 1
Instance 5:
Predicted label: 0
True label: 0
Instance 6:
Predicted label: 0
True label: 0
Instance 7:
Predicted label: 0
True label: 1
Instance 8:
Predicted label: 0
True label: 2
Instance 9:
Predicted label: 0
True label: 0
Instance 10:
Predicted label: 0
True label: 1
Instance 11:
Predicted label: 0
True label: 0
Instance 12:
Predicted label: 0
True label: 1
Instance 13:
Predicted label: 0
True label: 1
Instance 14:
Predicted label: 0
True label: 0
Instance 15:
Predicted label: 0
True label: 0
Instance 16:
Predicted label: 0
True label: 0
Instance 17:
Predicted label: 0
True label: 0
Instance 18:
Predicted label: 0
True label: 0
Instance 19:
Predicted label: 0
True label: 0
Instance 20:
Predicted label: 0
True label: 1
Instance 21:
Predicted label: 0
True label: 0
Instance 22:
Predicted label: 0
True label: